This tutorial will introduce you to  MongoDB a document database. MongoDB deals in data that is effictively a dictionary in Python or JSON data if it's a file. This document-oriented database is very different from the relational database designs we've seen thus far. We'll learn how to populate the database with data, how to query the database for documents we'd like to find, how to extract data for data engineering purposes.

In [1]:
!pip install faker
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 10.7 MB/s eta 0:00:00


In [2]:
from faker import Faker
from pymongo import MongoClient
import random
import datetime
import re

# install MongoDB

In [3]:
!apt-get install gnupg curl

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
curl is already the newest version (7.81.0-1ubuntu1.18).
gnupg is already the newest version (2.2.27-3ubuntu2.1).
gnupg set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [4]:
!curl -fsSL https://pgp.mongodb.com/server-7.0.asc | \
   sudo gpg -o /usr/share/keyrings/mongodb-server-7.0.gpg \
   --dearmor

In [5]:
!echo "deb [ arch=amd64,arm64 signed-by=/usr/share/keyrings/mongodb-server-7.0.gpg ] https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/7.0 multiverse" | sudo tee /etc/apt/sources.list.d/mongodb-org-7.0.list

deb [ arch=amd64,arm64 signed-by=/usr/share/keyrings/mongodb-server-7.0.gpg ] https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/7.0 multiverse


In [6]:
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Ign:3 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/7.0 InRelease
Get:4 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/7.0 Release [2,090 B]
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/7.0 Release.gpg [866 B]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,031 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,396 kB]
Get:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:13 https://repo.mongodb.org/ap

In [7]:
!apt-get install -y mongodb-org

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  mongodb-database-tools mongodb-mongosh mongodb-org-database mongodb-org-database-tools-extra
  mongodb-org-mongos mongodb-org-server mongodb-org-shell mongodb-org-tools
The following NEW packages will be installed:
  mongodb-database-tools mongodb-mongosh mongodb-org mongodb-org-database
  mongodb-org-database-tools-extra mongodb-org-mongos mongodb-org-server mongodb-org-shell
  mongodb-org-tools
0 upgraded, 9 newly installed, 0 to remove and 52 not upgraded.
Need to get 162 MB of archives.
After this operation, 581 MB of additional disk space will be used.
Get:1 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/7.0/multiverse amd64 mongodb-database-tools amd64 100.10.0 [46.4 MB]
Get:2 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/7.0/multiverse amd64 mongodb-mongosh amd64 2.3.2 [54.1 MB]
Get:3 https://repo.mongodb.org/apt/u

In [8]:
!mkdir /data
!mkdir /data/db

In [9]:
import subprocess
subprocess.Popen(["mongod"])

<Popen: returncode: None args: ['mongod']>

In [10]:
from pymongo import MongoClient
client = MongoClient()
client.list_database_names() # ['admin', 'local']

['admin', 'config', 'local']

## create the db

name our database

In [27]:
db = client['cloud_purchase_db']

## Set up Collections

drop the collections in case they already exist so we don't duplicate data

In [29]:
db.drop_collection('customers')
db.drop_collection('products')
#db.drop_collection('orders')

{'nIndexesWas': 1, 'ns': 'cloud_purchase_db.products', 'ok': 1.0}

do we have any collections?

## list collections

In [30]:
db.list_collection_names()

[]

In [14]:
# Create Faker instance
fake = Faker()

# Create the data

## create customer data

create from 1 to 5 orders (randomly)
pick a random product id from 1 to 10
units between 10 and 100
year from 1 year ago to today

In [15]:
# Generate customer orders
def gen_orders():

  orders = []
  for i in range(1,random.randint(1, 5)):

      productid = random.randint(1, 10)
      units = random.randint(10, 100)
      if random.random() < 0.1:
          units *= 10
      purchase_date = fake.date_between(start_date='-1y', end_date='today')
      purchase_date = datetime.datetime.combine(purchase_date, datetime.datetime.min.time())

      order = {
          'productid': productid,
          'units': units,
          'purchase_date': purchase_date
      }

      orders.append(order)

  return orders

In [16]:
gen_orders()

[]

generate 100 customers

In [17]:
# Generate customers
customers = []
for i in range(100):
    customer = {
        'customerid': i+1,
        'name': fake.name(),
        'email': fake.email(),
        'created' : datetime.datetime.combine(fake.date_between(start_date='-1y', end_date='today'), datetime.datetime.min.time()),
        'phone': [fake.phone_number(),fake.phone_number(),fake.phone_number()],
        'orders' : gen_orders() #this is where we generate orders
    }
    customers.append(customer)

In [18]:
len(customers)

100

In [19]:
customers[0:3]

[{'customerid': 1,
  'name': 'David Campbell',
  'email': 'amyrichardson@example.com',
  'created': datetime.datetime(2024, 3, 28, 0, 0),
  'phone': ['3618393744', '232.981.6227', '521.536.4875x30210'],
  'orders': [{'productid': 1,
    'units': 59,
    'purchase_date': datetime.datetime(2023, 11, 11, 0, 0)}]},
 {'customerid': 2,
  'name': 'Monica Kramer',
  'email': 'rubioalicia@example.com',
  'created': datetime.datetime(2024, 4, 23, 0, 0),
  'phone': ['214.459.0458x694', '457.378.5315x45776', '+1-740-698-5267x96491'],
  'orders': [{'productid': 8,
    'units': 19,
    'purchase_date': datetime.datetime(2024, 5, 6, 0, 0)},
   {'productid': 1,
    'units': 16,
    'purchase_date': datetime.datetime(2024, 3, 23, 0, 0)}]},
 {'customerid': 3,
  'name': 'Joshua Ruiz',
  'email': 'timothygarrett@example.net',
  'created': datetime.datetime(2024, 8, 29, 0, 0),
  'phone': ['729.281.5764', '(409)802-5381x74255', '(232)242-2537x4053'],
  'orders': [{'productid': 9,
    'units': 51,
    'purch

In [20]:
# Insert customers into MongoDB
db.customers.insert_many(customers)

InsertManyResult([ObjectId('6710582fe21a89bed3ac20a7'), ObjectId('6710582fe21a89bed3ac20a8'), ObjectId('6710582fe21a89bed3ac20a9'), ObjectId('6710582fe21a89bed3ac20aa'), ObjectId('6710582fe21a89bed3ac20ab'), ObjectId('6710582fe21a89bed3ac20ac'), ObjectId('6710582fe21a89bed3ac20ad'), ObjectId('6710582fe21a89bed3ac20ae'), ObjectId('6710582fe21a89bed3ac20af'), ObjectId('6710582fe21a89bed3ac20b0'), ObjectId('6710582fe21a89bed3ac20b1'), ObjectId('6710582fe21a89bed3ac20b2'), ObjectId('6710582fe21a89bed3ac20b3'), ObjectId('6710582fe21a89bed3ac20b4'), ObjectId('6710582fe21a89bed3ac20b5'), ObjectId('6710582fe21a89bed3ac20b6'), ObjectId('6710582fe21a89bed3ac20b7'), ObjectId('6710582fe21a89bed3ac20b8'), ObjectId('6710582fe21a89bed3ac20b9'), ObjectId('6710582fe21a89bed3ac20ba'), ObjectId('6710582fe21a89bed3ac20bb'), ObjectId('6710582fe21a89bed3ac20bc'), ObjectId('6710582fe21a89bed3ac20bd'), ObjectId('6710582fe21a89bed3ac20be'), ObjectId('6710582fe21a89bed3ac20bf'), ObjectId('6710582fe21a89bed3ac20

## create products data

In [21]:
# Generate products
products = []
for i in range(10):
    product = {
        'productid': i+1,
        'category': random.choice(['Electronics', 'Clothing', 'Books', 'Home']),
        'price': random.randint(1, 100)
    }
    products.append(product)

In [22]:
products[0:3]

[{'productid': 1, 'category': 'Clothing', 'price': 19},
 {'productid': 2, 'category': 'Home', 'price': 62},
 {'productid': 3, 'category': 'Home', 'price': 7}]

put the 10 products into the database

In [23]:
# Insert products into MongoDB
db.products.insert_many(products)

InsertManyResult([ObjectId('67105837e21a89bed3ac210b'), ObjectId('67105837e21a89bed3ac210c'), ObjectId('67105837e21a89bed3ac210d'), ObjectId('67105837e21a89bed3ac210e'), ObjectId('67105837e21a89bed3ac210f'), ObjectId('67105837e21a89bed3ac2110'), ObjectId('67105837e21a89bed3ac2111'), ObjectId('67105837e21a89bed3ac2112'), ObjectId('67105837e21a89bed3ac2113'), ObjectId('67105837e21a89bed3ac2114')], acknowledged=True)

In [24]:
db.list_collection_names() # list collections

['customers', 'products']

loop through collections and count the number of documents

In [25]:
for collection_name in db.list_collection_names():
  collection = db.get_collection(collection_name)
  print(f'{collection_name}:{collection.count_documents({})}')

customers:100
products:10


# Querying



## Customers

Find all customers.

In [26]:
for document in db.customers.find({}):
  print(document)

{'_id': ObjectId('6710582fe21a89bed3ac20a7'), 'customerid': 1, 'name': 'David Campbell', 'email': 'amyrichardson@example.com', 'created': datetime.datetime(2024, 3, 28, 0, 0), 'phone': ['3618393744', '232.981.6227', '521.536.4875x30210'], 'orders': [{'productid': 1, 'units': 59, 'purchase_date': datetime.datetime(2023, 11, 11, 0, 0)}]}
{'_id': ObjectId('6710582fe21a89bed3ac20a8'), 'customerid': 2, 'name': 'Monica Kramer', 'email': 'rubioalicia@example.com', 'created': datetime.datetime(2024, 4, 23, 0, 0), 'phone': ['214.459.0458x694', '457.378.5315x45776', '+1-740-698-5267x96491'], 'orders': [{'productid': 8, 'units': 19, 'purchase_date': datetime.datetime(2024, 5, 6, 0, 0)}, {'productid': 1, 'units': 16, 'purchase_date': datetime.datetime(2024, 3, 23, 0, 0)}]}
{'_id': ObjectId('6710582fe21a89bed3ac20a9'), 'customerid': 3, 'name': 'Joshua Ruiz', 'email': 'timothygarrett@example.net', 'created': datetime.datetime(2024, 8, 29, 0, 0), 'phone': ['729.281.5764', '(409)802-5381x74255', '(232

find customerid 76

In [ ]:
import pprint

In [ ]:
for document in db.customers.find({'customerid':76}):
  print(pprint.pprint(document))

In [ ]:
for document in db.customers.find({'$or':[{'customerid':76},{'customerid':6}]}):
  print(pprint.pprint(document))

In [ ]:
for document in db.customers.find({'customerid': {'$in': [76,6]}}):
  print(pprint.pprint(document))

find customers created in august 2023

In [ ]:
from datetime import datetime

In [ ]:
august_customers = db.customers.find({
    "created": {
        "$gte": datetime(2023, 8, 1),
        "$lt": datetime(2023, 9, 1)
    }
})

for customer in august_customers:
    print(customer)  # Or handle each document as needed

## Products

find all products

In [ ]:
for document in db.products.find({}):
  print(document)

find all products with prices less than 40

In [ ]:
for document in db.products.find({'price': {'$lt':40}}):
  print(document)

find all products with category clothing

the find method returns a cursor that lazily loads the result set in batches as we iterate over them.

In [ ]:
db.products.find({'category': 'Clothing'})

In [ ]:
for document in db.products.find({'category': 'Clothing'}):
  print(document)

not equal query

In [ ]:
for document in db.products.find({"category": {"$ne": "Clothing"}}):
  print(document)

the equivalent of SQL's query `like '%string%'`

In [ ]:
for document in db.products.find({"category": re.compile("electron", re.IGNORECASE)}):
  print(document)

find all products with price less than 40 AND category Clothing

In [ ]:
for document in db.products.find({'price': {'$lt':40},'category': 'Clothing'}):
  print(document)

## Orders

In [ ]:
# Calculate the total number of orders
pipeline = [
    {
        '$unwind': '$orders'
    },
    {
        '$group': {
            '_id': None,
            'total_orders': {'$sum': 1}
        }
    },
    {
        '$project': {
            '_id': 0,
            'total_orders': 1
        }
    }
]

result = db.customers.aggregate(pipeline)

# Extract the total number of orders
total_orders = next(result)['total_orders']

# Print the total number of orders
print(f"Total Orders: {total_orders}")

In [ ]:
filter_criteria = {
    'orders.units': {'$lt': 45000}
}

# Calculate the total number of orders with units less than 12000
pipeline = [
    {
      '$unwind': '$orders'
    },
    {
      '$match': filter_criteria
    },
    {
        '$group': {
            '_id': None,
            'total_orders': {'$sum': 1}
        }
    },
    {
        '$project': {
            '_id': 0,
            'total_orders': 1
        }
    }
]

result = db.customers.aggregate(pipeline)

# Extract the total number of orders
total_orders = next(result)['total_orders']

# Print the total number of orders
print(f"Total Orders: {total_orders}")

# Extraction


## Dump mongodb data to json file.

create a JSON file with the filtered data.

In [ ]:
from bson.json_util import dumps
import json

open a file. create a cursor that gets passed to dumps which takes binary json data and converts it serialized json, deseralizes it to python object and then serializes it back to a file.  

## dump orders

## dump customers

(and orders as they are nested)

In [ ]:
with open('customers.json', 'w') as file:
  cursor = db.customers.find({})
  file.write(dumps(cursor))

## dump products

In [ ]:
with open('products.json', 'w') as file:
  cursor = db.products.find({})
  file.write(dumps(cursor))

In [ ]:
!ls -lh *.json

# Reading JSON file data into Python

## JSON into Pandas DataFrame

In [ ]:
import pandas as pd

In [ ]:
customers_df = pd.read_json('customers.json')
customers_df.head(2)

In [ ]:
products_df = pd.read_json('products.json')
products_df.head(2)

### Merge DataFrames

In [ ]:
customers_df.head(4)

In [ ]:
orders_df = customers_df.explode('orders')

In [ ]:
df_orders_expanded = pd.concat([orders_df.drop(['orders'], axis=1), orders_df['orders'].apply(pd.Series)], axis=1)
df_orders_expanded.head(3)


In [ ]:
customer_product_orders = df_orders_expanded.merge(products_df, on='productid', how='left')
customer_product_orders.head(3)

In [ ]:
customer_product_orders['total_sales'] = customer_product_orders['price'] * customer_product_orders['units']

In [ ]:
customer_product_orders.groupby(by='category').agg({'total_sales': 'sum'}).sort_values(by='total_sales', ascending=False)

How to unzip a zip file.

In [ ]:
!wget -O patient_records.zip https://github.com/matthewpecsok/data_engineering/raw/main/data/patient_records.zip

In [ ]:
!ls -l

In [ ]:
!unzip patient_records.zip

In [ ]:
!wget -O medication_database.db https://github.com/matthewpecsok/data_engineering/raw/main/data/medication_database.db

In [ ]:
import sqlite3
import pandas as pd

In [ ]:
medication_con = sqlite3.connect('medication_database.db')

pd.read_sql_query('SELECT * FROM sqlite_master', medication_con)

In [ ]:
pd.read_sql_query('SELECT * FROM medications', medication_con)

In [ ]:
import json

In [ ]:
patients_1_batch = json.load(open('patient_records_batch_1.json'))

In [ ]:
len(patients_1_batch)

In [ ]:
patients_1_batch[0:20]